### Induced density maximisation

This note computes
$$
  \max_G t(\text{ind }C_5, G).
$$

Note that this is proven in [https://arxiv.org/abs/1411.4645] with optimal construction as iterative blow-up of $C_5$.
Due to the mismatch in the definition of density, one should multiply 12 ($v(C_5)! / |\mathrm{Aut}(C_5)|$) to the computed result to match the paper's value.

In [ ]:
import flag_algebra.flag_algebra as fa
import flag_algebra.build_problem as bp
import networkx as nx
import cvxpy as cp
import numpy as np
import itertools
import math

In [ ]:
# This code computes induced density of H in iterative blow-up of G.
def calculate_induced_density(H, G):
  m = len(H)
  k = len(G)

  densities = {}

  all_subsets = []
  for r in range(1, m+1):
    all_subsets.extend(itertools.combinations(range(m), r))
  
  for subset_tuple in all_subsets:
    subset = frozenset(subset_tuple)
    size = len(subset)

    if size == 1:
      densities[subset] = 1.0
      continue

    sum_split_val = 0.0
    ordered_subset = sorted(list(subset))
    
    for mapping in itertools.product(range(k), repeat=size):
      unique_buckets = set(mapping)
      
      if len(unique_buckets) == 1:
        continue
      
      consistent = True
      
      for i in range(size):
        for j in range(i + 1, size):
          u_idx = ordered_subset[i]
          v_idx = ordered_subset[j]
          
          bucket_u_idx = mapping[i]
          bucket_v_idx = mapping[j]
          
          if bucket_u_idx != bucket_v_idx:
            is_edge_H = H.has_edge(u_idx, v_idx)
            is_edge_G = G.has_edge(bucket_u_idx, bucket_v_idx)
            
            if is_edge_H != is_edge_G:
              consistent = False
              break
        if not consistent:
          break
      
      if consistent:
        term = 1.0
        
        bucket_groups = {b: [] for b in unique_buckets}
        for i, bucket_idx in enumerate(mapping):
          bucket_groups[bucket_idx].append(ordered_subset[i])
        
        for b_nodes in bucket_groups.values():
          if len(b_nodes) > 0:
            term *= densities[frozenset(b_nodes)]
        
        sum_split_val += term

    densities[subset] = sum_split_val / (k**size - k)

  prob_density = densities[frozenset(range(m))]
  
  return prob_density

density = calculate_induced_density(nx.cycle_graph(5), nx.cycle_graph(5))

In [ ]:
problem, variable_dict = bp.build_problem(
  objectives=[('ind', nx.cycle_graph(5), 1)], # Objective is induced-density of C5
  constraints=[], # No extra constraints
  sdp_configs=[(3, 1)], # Use SDPs from flags of size 3 with 1 labelled vertex
  lowerbound=False # We want to find the upper bound on the density
)

problem.solve(solver=cp.SCS)

print("Optimal value:", problem.value)
print("Induced density of C5 in iterated blow-up of C5", density)

In [ ]:
# Same problem, but with stronger SDPs
problem, variable_dict = bp.build_problem(
  objectives=[('ind', nx.cycle_graph(5), 1)],
  constraints=[],
  sdp_configs=[(4, 1), (5, 3), (6, 5)],
  lowerbound=False
)

problem.solve(solver=cp.SCS)

print("Optimal value:", problem.value)
print("Induced density of C5 in iterated blow-up of C5", density)